In [ ]:
import geopandas as gpd
import rasterio as rio

import rasterstats

import matplotlib.pyplot as plt
from rasterio.plot import show

from os.path import join, basename
from glob import glob

In [ ]:
ROOT_DIR = '/media/sf_JD/DP'
stats = ['mean', 'median', 'std']

## Testing raster statistics

In [ ]:
def compute_zonal_stats_one_orbit(raster_dir, vector_path, stats=['mean']):
    # Get relative orbit and raster filenames
    ro = basename(raster_dir)
    rasters_list = glob('coh_*.tif', root_dir=raster_dir)
    # Read vector file
    stats_gdf = gpd.read_file(vector_path)
    
    print(f'Processing {len(rasters_list)} rasters:')
    # loop through all coherence rasters
    for raster in rasters_list:
        print(raster)
        
        # Read raster array and transform
        raster_path = join(raster_dir, raster)
        with rio.open(raster_path) as src:
            arr = src.read()
            affine = src.transform
        
        # compute zonal statistics for individual polarizations
        for idx_band, pol in enumerate(('VH', 'VV')):
            prefix = f'{ro:03}_{pol}_{raster[4:8]}_{raster[8:12]}-{raster[17:21]}_'

            stats_list = rasterstats.zonal_stats(stats_gdf, arr[idx_band,:,:], affine=affine, geojson_out=True, prefix=prefix, stats=stats, nodata=-1)
            stats_gdf = gpd.GeoDataFrame.from_features(stats_list, crs=stats_gdf.crs)

    print('Zonal stats successfuly computed.')
    return stats_gdf

In [ ]:
def compute_zonal_stats_all_orbits(root_dir, year, stats):
    
    gdf_vector = join(root_dir, f'reference/{year}/vyjezdy_{year}_4326_singlepart.gpkg')
    out_vector = join(root_dir, f'reference/{year}/vyjezdy_{year}_zonal_stats.gpkg')
    raster_dir = join(root_dir, f's1_coherence/{year}')

    for ro_dir in glob(f'{raster_dir}/*'):
        print(ro_dir)
        gdf_vector = compute_zonal_stats_one_orbit(ro_dir, gdf_vector, stats)
    
    gdf_vector.to_file(out_vector)
    return gdf_vector

In [ ]:
def zonal_stats_all(root_dir, years=(2021,), stats=['mean']):
    for year in years:
        compute_zonal_stats_all_orbits(root_dir, year, stats)

In [ ]:
zonal_stats_all(ROOT_DIR, years=(2021,), stats=stats)

## possible to delete:

In [ ]:
gdf_out = compute_zonal_stats_one_orbit(raster_dir, inpath_vector, stats)
gdf_out.to_file(outpath_vector)

In [ ]:
for raster in rasters_list:
    raster_path = join(raster_dir, raster)
    prefix = f'{raster[4:8]}_{raster[8:12]}-{raster[17:21]}_'
    
    with rio.open(raster_path) as src:
        for idx, polarization in enumerate(('VH', 'VV')):
            prefix_pol = f'{polarization}_{prefix}'
            print(prefix_pol)
    
            stats_list = rasterstats.zonal_stats(stats_gdf, src.read(idx+1), affine=src.transform, geojson_out=True, prefix=prefix_pol, stats=stats, nodata=-1)
            stats_gdf = gpd.GeoDataFrame.from_features(stats_list, crs='EPSG:4326')

In [ ]:
stats_gdf.to_file(outpath_vector)

In [ ]:
inpath_raster = 'c:/users/dd/documents/natur_cuni/_dp/DTM/DMR_4G_4326.tif'
inpath_vector = 'c:/users/dd/documents/natur_cuni/_dp/reference_data/Vyjezdy_2021/vyjezdy_2021_4326_singlepart.gpkg'
outpath_vector = 'c:/users/dd/documents/natur_cuni/_dp/reference_data/Vyjezdy_2021/vyjezdy_2021_stats.gpkg'

In [ ]:
stats_list = rasterstats.zonal_stats(inpath_vector, inpath_raster, prefix='dtm_', geojson_out=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

with rio.open(inpath_raster) as src:
    show(src.read(1), transform=src.transform, cmap='pink', ax=ax)

stats_gdf = gpd.GeoDataFrame.from_features(stats_list, crs="EPSG:4326")
stats_gdf.plot('dtm_max', ax=ax, figsize=(30, 30))
#gpd.read_file(inpath_vector).combine(pd.DataFrame(stats_list))

In [ ]:
inpath_vector = 'c:/users/dd/documents/natur_cuni/_dp/reference_data/Vyjezdy_2021/vyjezdy_2021_4326_singlepart.gpkg'
gdf = gpd.read_file(inpath_vector)

In [ ]:
gdf.crs